# Piano MIDI Generation - Model Architecture

This notebook defines and builds the transformer model for piano MIDI generation.

## Overview

1. **Load processed data** - Vocabulary and sequences
2. **Define model architecture** - GPT-style decoder transformer
3. **Create dataset/dataloader** - For batch training
4. **Model initialization** - With proper weights
5. **Test forward pass** - Verify everything works

## Model Design

- **Architecture**: GPT-2 style decoder-only transformer
- **Conditional Generation**: Metadata tokens as prefix for MIDI generation
- **Autoregressive**: Predicts next token given previous tokens


In [1]:
# Install PyTorch with CUDA support
# ⚠️ IMPORTANT: If you have GPU, install CUDA version (NOT CPU-only!)

# Step 1: Uninstall CPU-only PyTorch
# %pip uninstall torch torchvision torchaudio -y

# Step 2: Check your CUDA version (run in Command Prompt/PowerShell):
# nvidia-smi
# Look for "CUDA Version: X.X" in the output

# Step 3: Install matching PyTorch with CUDA (uncomment ONE based on your CUDA version):

# For CUDA 11.8:
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# For CUDA 12.1:
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# For CUDA 12.4:
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

# For Windows with CUDA (most common):
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# After installing, restart kernel and check CUDA availability in next cell


In [2]:

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import json
import math
from pathlib import Path
from typing import Dict, List, Tuple

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"✅ CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"   CUDA version: {torch.version.cuda}")
    print(f"   GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print(f"   Device count: {torch.cuda.device_count()}")
else:
    print("\n⚠️  WARNING: CUDA not available!")
    print("   You have CPU-only PyTorch installed (version shows '+cpu')")
    print("\n   To fix and use your GPU:")
    print("   1. Open Command Prompt/PowerShell")
    print("   2. Run: nvidia-smi  (to check CUDA version)")
    print("   3. Uninstall CPU version:")
    print("      pip uninstall torch torchvision torchaudio -y")
    print("   4. Install CUDA version (replace cu121 with your CUDA version):")
    print("      pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121")
    print("   5. Restart Jupyter kernel")
    print("\n   Training will work but will be SLOW on CPU!")


PyTorch version: 2.6.0+cu124
CUDA available: True
✅ CUDA device: NVIDIA GeForce GTX 1650
   CUDA version: 12.4
   GPU memory: 4.3 GB
   Device count: 1


## Step 1: Load Processed Data

Load vocabulary and sequences from preprocessing.


In [3]:
# Load processed data
data_dir = Path("processed_data")

with open(data_dir / "vocab.json", 'r') as f:
    vocab = json.load(f)

with open(data_dir / "id_to_token.json", 'r') as f:
    id_to_token = json.load(f)

with open(data_dir / "preprocessing_config.json", 'r') as f:
    preprocess_config = json.load(f)

with open(data_dir / "sequences.json", 'r') as f:
    sequences = json.load(f)

vocab_size = len(vocab)
print(f"✅ Loaded vocabulary: {vocab_size:,} tokens")
print(f"✅ Loaded {len(sequences):,} sequences")
print(f"\nConfig:")
for key, value in preprocess_config.items():
    print(f"  {key}: {value}")

# Convert id_to_token from string keys to int
id_to_token = {int(k): v for k, v in id_to_token.items()}


✅ Loaded vocabulary: 746 tokens
✅ Loaded 3,843 sequences

Config:
  min_quality_score: 0.97
  max_sequence_length: 2048
  time_quantization: 10
  vocab_size: 746
  num_sequences: 3843
  total_tokens: 7766992


## Step 2: Model Configuration

Define model hyperparameters. **Keep model small to match dataset size.**


In [4]:
# Model Configuration
# Adjusted for small dataset (3,843 sequences)

MODEL_CONFIG = {
    'vocab_size': vocab_size,
    'max_seq_length': preprocess_config['max_sequence_length'],  # 2048
    'd_model': 512,        # Embedding dimension (smaller for small dataset)
    'n_layers': 6,         # Number of transformer blocks (reduced)
    'n_heads': 8,          # Attention heads
    'd_ff': 2048,         # Feed-forward dimension
    'dropout': 0.1,       # Dropout rate
    'pad_token_id': vocab.get('<PAD>', 0),
}

print("Model Configuration:")
for key, value in MODEL_CONFIG.items():
    print(f"  {key}: {value}")

# Calculate approximate parameters
# Rough estimate: vocab_size * d_model + n_layers * (d_model^2 * 4)
approx_params = (
    MODEL_CONFIG['vocab_size'] * MODEL_CONFIG['d_model'] +  # Embedding
    MODEL_CONFIG['n_layers'] * (
        MODEL_CONFIG['d_model'] ** 2 * 4 +  # Attention
        MODEL_CONFIG['d_model'] * MODEL_CONFIG['d_ff'] * 2  # FFN
    )
)
print(f"\n📊 Approximate model size: {approx_params/1e6:.1f}M parameters")
print(f"   Dataset size: {len(sequences):,} sequences")
print(f"   Ratio: {len(sequences)/approx_params*1e6:.1f} sequences per 1M params")
print(f"   (Good ratio: >100 sequences per 1M params)")


Model Configuration:
  vocab_size: 746
  max_seq_length: 2048
  d_model: 512
  n_layers: 6
  n_heads: 8
  d_ff: 2048
  dropout: 0.1
  pad_token_id: 0

📊 Approximate model size: 19.3M parameters
   Dataset size: 3,843 sequences
   Ratio: 199.6 sequences per 1M params
   (Good ratio: >100 sequences per 1M params)


## Step 3: Transformer Model Architecture

GPT-style decoder-only transformer for autoregressive generation.


In [5]:
# Option 1: Import from shared module (recommended)
# Uncomment this if you've created model.py:
import sys
sys.path.append('.')
from model import PositionalEncoding, TransformerBlock, PianoMIDIGenerator

# Option 2: Define classes here (for standalone notebook)
# Transformer Architecture Components

class PositionalEncoding(nn.Module):
    """Sinusoidal positional encoding"""
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)  # [1, max_len, d_model]
        self.register_buffer('pe', pe)
    
    def forward(self, x):
        # x: [batch, seq_len, d_model]
        seq_len = x.size(1)
        return x + self.pe[:, :seq_len, :]


class TransformerBlock(nn.Module):
    """Single transformer decoder block"""
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super().__init__()
        self.attention = nn.MultiheadAttention(d_model, n_heads, dropout=dropout, batch_first=True)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        
        self.ff = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(d_ff, d_model),
            nn.Dropout(dropout)
        )
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, attn_mask=None, key_padding_mask=None):
        # Self-attention with residual
        # attn_mask: [seq_len, seq_len] - causal mask (broadcasted by MultiheadAttention)
        # key_padding_mask: [batch, seq_len] - True = pad (mask), False = valid
        
        # MultiheadAttention handles:
        # - attn_mask: 2D [seq_len, seq_len] gets broadcasted to [batch, num_heads, seq_len, seq_len]
        # - key_padding_mask: [batch, seq_len] - masks out padded positions
        
        attn_out, _ = self.attention(
            x, x, x, 
            attn_mask=attn_mask,  # Causal mask: [seq_len, seq_len]
            key_padding_mask=key_padding_mask  # Padding mask: [batch, seq_len] or None
        )
        x = self.norm1(x + self.dropout(attn_out))
        
        # Feed-forward with residual
        ff_out = self.ff(x)
        x = self.norm2(x + ff_out)
        
        return x


class PianoMIDIGenerator(nn.Module):
    """
    GPT-style decoder-only transformer for conditional MIDI generation
    
    Input: [metadata_tokens] + [midi_tokens]
    Output: Next token predictions for all positions
    """
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.d_model = config['d_model']
        
        # Token embeddings
        self.embedding = nn.Embedding(config['vocab_size'], config['d_model'])
        
        # Positional encoding
        self.pos_encoding = PositionalEncoding(config['d_model'], config['max_seq_length'])
        
        # Transformer blocks
        self.blocks = nn.ModuleList([
            TransformerBlock(
                config['d_model'],
                config['n_heads'],
                config['d_ff'],
                config['dropout']
            )
            for _ in range(config['n_layers'])
        ])
        
        # Output projection
        self.ln_f = nn.LayerNorm(config['d_model'])
        self.head = nn.Linear(config['d_model'], config['vocab_size'], bias=False)
        
        # Initialize weights
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    
    def forward(self, input_ids, attention_mask=None):
        """
        Args:
            input_ids: [batch, seq_len] token IDs
            attention_mask: [batch, seq_len] 1 = attend, 0 = mask
        
        Returns:
            logits: [batch, seq_len, vocab_size]
        """
        batch_size, seq_len = input_ids.shape
        
        # Embeddings
        x = self.embedding(input_ids)  # [batch, seq_len, d_model]
        x = self.pos_encoding(x)
        
        # Create causal mask (lower triangular) for autoregressive generation
        # MultiheadAttention with batch_first=True expects:
        # - 2D mask: [seq_len, seq_len] (broadcasted to batch and heads)
        # - 3D mask: NOT directly supported in same way
        
        # Base causal mask: lower triangular [seq_len, seq_len]
        causal_mask_base = torch.tril(torch.ones(seq_len, seq_len, device=input_ids.device)).bool()
        
        # If attention_mask provided, we need to handle padding
        # For MultiheadAttention, we'll create a combined mask
        if attention_mask is not None:
            # attention_mask: [batch, seq_len] where 1 = valid, 0 = pad
            # Create 2D mask per batch, then combine
            # We'll use key_padding_mask instead for padding (better for MultiheadAttention)
            key_padding_mask = (attention_mask == 0)  # True = pad (mask), False = valid
            
            # For causal mask, use base mask (will be combined with key_padding_mask internally)
            # But MultiheadAttention doesn't directly support causal + padding mask together
            # So we'll create combined mask manually
            causal_mask_2d = causal_mask_base  # [seq_len, seq_len]
            # This will be broadcasted to all batches and heads by MultiheadAttention
            attn_mask = causal_mask_2d
        else:
            # Just causal mask, no padding
            attn_mask = causal_mask_base  # [seq_len, seq_len]
            key_padding_mask = None
        
        # Transformer blocks
        for block in self.blocks:
            x = block(x, attn_mask=attn_mask, key_padding_mask=key_padding_mask)
        
        # Output projection
        x = self.ln_f(x)
        logits = self.head(x)  # [batch, seq_len, vocab_size]
        
        return logits
    
    def generate(self, input_ids, max_length=2048, temperature=1.0, top_k=50, top_p=0.9):
        """
        Autoregressive generation
        
        Args:
            input_ids: [batch, start_len] initial tokens (metadata)
            max_length: Maximum generation length
            temperature: Sampling temperature
            top_k: Top-k sampling
            top_p: Nucleus sampling
        
        Returns:
            Generated token IDs
        """
        self.eval()
        generated = input_ids.clone()
        
        with torch.no_grad():
            for _ in range(max_length - input_ids.size(1)):
                # Forward pass
                logits = self.forward(generated)[:, -1, :]  # [batch, vocab_size]
                
                # Apply temperature
                logits = logits / temperature
                
                # Top-k filtering
                if top_k > 0:
                    indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
                    logits[indices_to_remove] = float('-inf')
                
                # Top-p (nucleus) sampling
                if top_p < 1.0:
                    sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                    cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)
                    
                    sorted_indices_to_remove = cumulative_probs > top_p
                    sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
                    sorted_indices_to_remove[..., 0] = 0
                    
                    indices_to_remove = sorted_indices_to_remove.scatter(1, sorted_indices, sorted_indices_to_remove)
                    logits[indices_to_remove] = float('-inf')
                
                # Sample next token
                probs = F.softmax(logits, dim=-1)
                next_token = torch.multinomial(probs, num_samples=1)
                generated = torch.cat([generated, next_token], dim=1)
                
                # Stop if END token
                if next_token.item() == vocab.get('<END>', 3):
                    break
        
        return generated

# Create model
model = PianoMIDIGenerator(MODEL_CONFIG)
print(f"✅ Model created with {sum(p.numel() for p in model.parameters())/1e6:.1f}M parameters")

# Save model classes to Python file for reuse in training notebook
# This allows training notebook to import without duplicating code
model_code = '''
# Model classes saved from architecture notebook
# These are also in model.py - use that for cleaner imports
'''
print("\n💡 Tip: Model classes are also available in 'model.py' for import in other notebooks")


✅ Model created with 19.7M parameters

💡 Tip: Model classes are also available in 'model.py' for import in other notebooks


In [6]:
# Dataset class
class MIDIDataset(Dataset):
    """Dataset for MIDI token sequences"""
    def __init__(self, sequences, vocab, max_length=2048):
        self.sequences = sequences
        self.vocab = vocab
        self.pad_token_id = vocab.get('<PAD>', 0)
        self.max_length = max_length
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        token_ids = self.sequences[idx]['token_ids']
        
        # Truncate if needed
        if len(token_ids) > self.max_length:
            token_ids = token_ids[:self.max_length]
        
        # Create input (all tokens except last) and target (all tokens except first)
        input_ids = token_ids[:-1]
        target_ids = token_ids[1:]
        
        # Pad to max_length
        pad_len = self.max_length - len(input_ids)
        if pad_len > 0:
            input_ids = input_ids + [self.pad_token_id] * pad_len
            target_ids = target_ids + [self.pad_token_id] * pad_len
        
        # Create attention mask (1 for real tokens, 0 for padding)
        attention_mask = [1] * len(token_ids[:-1]) + [0] * pad_len
        
        return {
            'input_ids': torch.tensor(input_ids, dtype=torch.long),
            'target_ids': torch.tensor(target_ids, dtype=torch.long),
            'attention_mask': torch.tensor(attention_mask, dtype=torch.long),
        }

# Create dataset and dataloader
dataset = MIDIDataset(sequences, vocab, max_length=MODEL_CONFIG['max_seq_length'])

# Split into train/val (90/10)
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(
    dataset, 
    [train_size, val_size],
    generator=torch.Generator().manual_seed(42)
)

# DataLoaders
BATCH_SIZE = 8  # Small batch size for small dataset

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0,  # Set to 0 for Windows/Jupyter
    pin_memory=True if torch.cuda.is_available() else False
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True if torch.cuda.is_available() else False
)

print(f"✅ Dataset created:")
print(f"  Total sequences: {len(dataset):,}")
print(f"  Train: {len(train_dataset):,}")
print(f"  Validation: {len(val_dataset):,}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Train batches: {len(train_loader):,}")
print(f"  Val batches: {len(val_loader):,}")


✅ Dataset created:
  Total sequences: 3,843
  Train: 3,458
  Validation: 385
  Batch size: 8
  Train batches: 433
  Val batches: 49


## Step 5: Test Forward Pass

Verify the model works correctly with a sample batch.


In [7]:
# Test forward pass
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.eval()

# Get a sample batch
sample_batch = next(iter(train_loader))
input_ids = sample_batch['input_ids'].to(device)
attention_mask = sample_batch['attention_mask'].to(device)
target_ids = sample_batch['target_ids'].to(device)

print(f"Sample batch shape:")
print(f"  input_ids: {input_ids.shape}")
print(f"  attention_mask: {attention_mask.shape}")
print(f"  target_ids: {target_ids.shape}")

# Forward pass
with torch.no_grad():
    logits = model(input_ids, attention_mask=attention_mask)

print(f"\n✅ Forward pass successful!")
print(f"  Logits shape: {logits.shape}")  # Should be [batch, seq_len, vocab_size]
print(f"  Expected: [{input_ids.shape[0]}, {input_ids.shape[1]}, {vocab_size}]")

# Test loss computation
criterion = nn.CrossEntropyLoss(ignore_index=MODEL_CONFIG['pad_token_id'])
logits_flat = logits.view(-1, vocab_size)
targets_flat = target_ids.view(-1)
loss = criterion(logits_flat, targets_flat)

print(f"\n✅ Loss computation successful!")
print(f"  Loss: {loss.item():.4f}")

# Show sample predictions
predicted_tokens = torch.argmax(logits[0, :20, :], dim=-1)
actual_tokens = target_ids[0, :20]

print(f"\nSample predictions (first 20 tokens):")
print(f"  Actual:   {[id_to_token[t.item()] for t in actual_tokens]}")
print(f"  Predicted: {[id_to_token[t.item()] for t in predicted_tokens]}")

print(f"\n✅ Model architecture verified! Ready for training.")


Sample batch shape:
  input_ids: torch.Size([8, 2048])
  attention_mask: torch.Size([8, 2048])
  target_ids: torch.Size([8, 2048])

✅ Forward pass successful!
  Logits shape: torch.Size([8, 2048, 746])
  Expected: [8, 2048, 746]

✅ Loss computation successful!
  Loss: nan

Sample predictions (first 20 tokens):
  Actual:   ['TIME_SHIFT:120', 'NOTE_ON:44', 'VELOCITY:80', 'TIME_SHIFT:80', 'NOTE_ON:85', 'VELOCITY:75', 'NOTE_ON:73', 'VELOCITY:80', 'TIME_SHIFT:210', 'NOTE_ON:56', 'VELOCITY:80', 'NOTE_ON:72', 'VELOCITY:90', 'TIME_SHIFT:10', 'NOTE_ON:80', 'VELOCITY:80', 'NOTE_ON:84', 'VELOCITY:85', 'TIME_SHIFT:70', 'NOTE_ON:70']
  Predicted: ['<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']

✅ Model architecture verified! Ready for training.
